In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from transformers import BertTokenizer

In [2]:
df=pd.read_excel('dataset_filledsupplier_currency_orderday.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24621 entries, 0 to 24620
Data columns (total 32 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   청구서번호        24621 non-null  object 
 1   No.          24621 non-null  int64  
 2   Subject      24599 non-null  object 
 3   Machinery    24621 non-null  object 
 4   Assembly     24621 non-null  object 
 5   청구품목         24621 non-null  object 
 6   Unnamed: 6   0 non-null      float64
 7   Part No.1    24602 non-null  object 
 8   Part No.2    3592 non-null   object 
 9   청구량          24517 non-null  float64
 10  견적           24171 non-null  object 
 11  견적수량         24517 non-null  float64
 12  견적화폐         24621 non-null  object 
 13  견적단가         24621 non-null  float64
 14  발주번호         24621 non-null  object 
 15  발주처          24621 non-null  object 
 16  발주           24621 non-null  object 
 17  발주수량         24621 non-null  int64  
 18  발주금액         24621 non-null  float64
 19  D/T 

In [4]:
print(len(df['발주처'].unique()))

81


## 클리닝

In [5]:
missing_conditions = df[
    df['발주'].notnull() &  # 발주 일자는 비어있지 않음
    df['미입고 기간'].isnull() &  # 미입고 기간은 비어있음
    df['창고입고'].isnull() & # 창고 입고도 비어있음
    df['선박입고'].isnull()  # 선박 입고도 비어있음

]

print(f"발주 일자는 있지만 미입고 기간, 창고 입고, 선박 입고도 없는 경우: {len(missing_conditions)}개")
df = df.drop(missing_conditions.index)

print(f"삭제된 행의 개수: {len(missing_conditions)}개")
print(f"남은 데이터프레임의 크기: {df.shape}")

발주 일자는 있지만 미입고 기간, 창고 입고, 선박 입고도 없는 경우: 1699개
삭제된 행의 개수: 1699개
남은 데이터프레임의 크기: (22922, 32)


In [6]:
#미입고기간으로 처리.
missing_both = df[df['창고입고'].isnull() & df['미입고 기간'].notnull()]

print(f"창고 입고일은 없고 미입고 기간은 명시되어 있어 미입고 기간으로 분류해야 할 경우 : {len(missing_both)}개")

창고 입고일은 없고 미입고 기간은 명시되어 있어 미입고 기간으로 분류해야 할 경우 : 1620개


In [7]:
df = df[df['미입고 기간'].isnull()]

df['발주'] = pd.to_datetime(df['발주'], errors='coerce')
df['창고입고'] = pd.to_datetime(df['창고입고'], errors='coerce')

# 리드타임 계산
df['리드타임'] = (df['창고입고'] - df['발주']).dt.days
df['리드타임'] = df['리드타임'].apply(lambda x: 1 if x == 0 else x)

In [8]:

df = df[(df['리드타임'] > 1 ) & (df['리드타임'] < 150)]
df = df.dropna(subset=['창고입고'])

In [9]:
print(df[['발주', '창고입고']].head(), df['창고입고'].isnull().sum())

          발주       창고입고
0 2019-01-11 2019-05-03
1 2019-01-11 2019-04-18
2 2019-01-11 2019-05-03
5 2019-06-03 2019-06-15
6 2019-06-03 2019-06-15 0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17520 entries, 0 to 24602
Data columns (total 33 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   청구서번호        17520 non-null  object        
 1   No.          17520 non-null  int64         
 2   Subject      17509 non-null  object        
 3   Machinery    17520 non-null  object        
 4   Assembly     17520 non-null  object        
 5   청구품목         17520 non-null  object        
 6   Unnamed: 6   0 non-null      float64       
 7   Part No.1    17505 non-null  object        
 8   Part No.2    2528 non-null   object        
 9   청구량          17455 non-null  float64       
 10  견적           17288 non-null  object        
 11  견적수량         17455 non-null  float64       
 12  견적화폐         17520 non-null  object        
 13  견적단가         17520 non-null  float64       
 14  발주번호         17520 non-null  object        
 15  발주처          17520 non-null  object        
 16  발주       

### 날짜뽑기

> 연도는 숫자형으로 처리(새로운 연도에 대응)
> 나머지는 원핫인코딩

In [11]:
def remove_restricted_text(text):
    return re.sub(r'\(사용금지\)', '', text).strip()

# Remove "(사용금지)" from '청구품목' and '발주처' columnsdf['청구품목'] = df['청구품목'].apply(remove_restricted_text)
df['발주처'] = df['발주처'].apply(remove_restricted_text)

In [13]:
past_columns = [
    'Machinery', 'Assembly', '청구품목', 'Part No.1', '견적화폐', '견적단가', '발주처', '발주', '창고입고', '리드타임'
]

# 해당 컬럼들만 선택하여 새로운 데이터프레임 생성
df_past = df[past_columns]

In [14]:
df_past_normalized = df_past.apply(lambda x: x.str.strip().str.upper() if x.dtype == "object" else x)


In [15]:
# 1. Machinery 테이블 생성
machinery_df = df_past_normalized[['Machinery']].drop_duplicates().reset_index(drop=True)
machinery_df['machinery_id'] = machinery_df.index + 1  # machinery_id 생성

In [16]:
# 2. Assembly 테이블 생성 및 Machinery와의 관계 설정 (고유한 assembly_id 설정)
assembly_df = df_past_normalized[['Assembly']].drop_duplicates().reset_index(drop=True)
assembly_df['assembly_id'] = assembly_df.index + 1  # 고유 assembly_id 생성

In [17]:
# Machinery와 Assembly를 매핑하여 assembly_df 확장 (machinery_id 추가)
assembly_df = pd.merge(df_past_normalized[['Machinery', 'Assembly']].drop_duplicates(), assembly_df, on='Assembly', how='left')
assembly_df = pd.merge(assembly_df, machinery_df, on='Machinery', how='left')

In [18]:
assembly_df['machinery_assembly_id'] = assembly_df.index + 1


In [17]:
# 3. 청구품목 등 상세 테이블 생성 및 병합
details_df = df_past_normalized[['Machinery', 'Assembly', '청구품목', 'Part No.1', '견적화폐', '견적단가', '발주처', '발주', '창고입고', '리드타임']]
details_df = pd.merge(details_df, assembly_df, on=['Machinery', 'Assembly'], how='left')  # assembly_id와 machinery_id 추가


In [18]:
# 발주와 창고입고가 동일한 경우 중복 제거 (하나만 살리기)
group_columns = ['machinery_id', 'assembly_id', '청구품목', 'Part No.1', '견적화폐', '견적단가', '발주처']
details_df = details_df.drop_duplicates(subset=group_columns + ['발주', '창고입고'], keep='first')



In [21]:
# 동일한 그룹에서 '발주'와 '창고입고'가 서로 다른 경우는 모두 살려둠
grouped = details_df.groupby(group_columns)
details_1002 = grouped.filter(lambda x: len(x[['발주', '창고입고']].drop_duplicates()) > 1)



In [22]:
machinery_df.to_csv('machinery_1002.csv', index=False, quotechar='"', quoting=csv.QUOTE_MINIMAL)
assembly_df.to_csv('assembly_1002.csv', index=False, quotechar='"', quoting=csv.QUOTE_MINIMAL)
details_1002.to_csv('details_1002.csv', index=False, quotechar='"', quoting=csv.QUOTE_MINIMAL)

In [19]:
columns_1002 = [
    'Machinery', 'Assembly', '청구품목', 'Part No.1', '견적화폐', '견적단가', '발주처'
]

# 해당 컬럼들만 선택하여 새로운 데이터프레임 생성
df_1002 = df[columns_1002]

In [20]:
# 양쪽 공백만 제거한 데이터프레임 생성 (중간 공백은 유지)
df_1002_normalized = df_1002.apply(lambda x: x.str.strip().str.upper() if x.dtype == "object" else x)


In [21]:
# 3. 청구품목 등 상세 테이블 생성 및 병합
details_for_item = df_1002_normalized[['Machinery', 'Assembly', '청구품목', 'Part No.1', '견적화폐', '견적단가', '발주처']]
details_for_item = pd.merge(details_for_item, assembly_df, on=['Machinery', 'Assembly'], how='left')  # assembly_id와 machinery_id 추가


In [22]:
# 기존의 details_for_item에 machinery_assembly_id 추가
details_for_item = pd.merge(details_for_item, assembly_df[['Machinery', 'Assembly', 'machinery_assembly_id']], on=['Machinery', 'Assembly'], how='left')


In [23]:
# 모든 컬럼을 기준으로 그룹화하여 중복이 없는 항목들만 남기기
group_columns = ['machinery_id', 'assembly_id', '청구품목', 'Part No.1', '견적화폐', '견적단가', '발주처']
# 그룹 크기가 1인 항목만 필터링
details_for_item_unique = details_for_item.groupby(group_columns).filter(lambda x: len(x) == 1)


In [24]:
# 최종 결과 확인 및 CSV 파일 생성
details_for_item_unique.to_csv('item_1002_v0.2.csv', index=False, encoding='utf-8-sig')


In [25]:

assembly_df.to_json('assembly_1002_v0.2.json', orient='records', force_ascii=False)

In [26]:
assembly_df.to_csv('assembly_1002_v0.csv', index=False, quotechar='"', quoting=csv.QUOTE_MINIMAL)


In [44]:
# 최종 데이터프레임을 json 파일로 저장
machinery_df.to_json('machinery1001.json', orient='records', force_ascii=False)


In [46]:
# CSV로 저장
machinery_df.to_csv('machinery_df.csv', index=False, quotechar='"', quoting=csv.QUOTE_MINIMAL)
assembly_df.to_csv('assembly_df.csv', index=False, quotechar='"', quoting=csv.QUOTE_MINIMAL)
details_df_diff_dates.to_csv('details_df_diff_dates.csv', index=False, quotechar='"', quoting=csv.QUOTE_MINIMAL)